# 🎯 Prédictions Ligue 1 - Approche Simple

Ce notebook utilise une méthode de prédiction **simple et transparente** basée sur un score composite.

## ⚠️ Prérequis Important

**Avant d'utiliser ce notebook**, vous devez d'abord exécuter :

### 📥 `0_collecte_donnees.ipynb`

Ce notebook collecte et met en cache toutes les données nécessaires (statistiques, classement, etc.).

**Sans cette étape**, ce notebook ne pourra pas fonctionner car il utilise **uniquement les données du cache** (aucun appel API direct).

---

## 🎯 Méthodologie

### Score Composite

$$\text{Score} = \text{Forme} + (\text{Buts marqués} - \text{Buts encaissés}) \times 0.5$$

**Composantes :**
- **Forme récente** : W=3, D=1, L=0 (maximum 15 points pour 5 matchs)
- **Différence de buts pondérée** : Coefficient 0.5 pour équilibrer

### Seuil de Décision

- Différence > 2 points → Victoire probable
- Différence ≤ 2 points → Match serré

### Caractéristiques

✅ **Simple** : Formule transparente  
✅ **Rapide** : Calcul instantané  
✅ **Sans API** : Pas d'appels API dans ce notebook  
❌ **Limites** : Ne prend pas en compte domicile/extérieur  

📖 **[Documentation complète de la méthode](docs/prediction-simple.md)**

## 📋 Prérequis

1. Obtenir une clé API gratuite sur [API-Football](https://www.api-football.com/) (100 requêtes/jour)
2. Installer les dépendances nécessaires

In [1]:
# Installation des dépendances (à exécuter une seule fois)
# !pip install requests pandas numpy scikit-learn python-dotenv

import requests
import json
from datetime import datetime, timedelta
import pandas as pd
import numpy as np

In [2]:
# Configuration - Chargement depuis .env
import os
from dotenv import load_dotenv

# Charger les variables d'environnement depuis le fichier .env
load_dotenv()

API_KEY = os.getenv("API_FOOTBALL_KEY", "votre_clé_api_ici")  # ⚠️ Configurez dans .env
LIGUE1_ID = int(os.getenv("LIGUE1_ID", 61))
CURRENT_SEASON = int(os.getenv("CURRENT_SEASON", 2025))

print(f"✅ Configuration chargée : Ligue {LIGUE1_ID}, Saison {CURRENT_SEASON}")
print(f"📅 Saison {CURRENT_SEASON}-{CURRENT_SEASON+1}")
if API_KEY == "votre_clé_api_ici":
    print("⚠️ N'oubliez pas de configurer votre API_FOOTBALL_KEY dans le fichier .env")

✅ Configuration chargée : Ligue 61, Saison 2025
📅 Saison 2025-2026


### 📅 Note sur les saisons

L'API-Football utilise l'**année de début** de saison :
- Saison 2025-2026 (en cours) → `CURRENT_SEASON = 2025`
- Saison 2024-2025 (terminée) → `CURRENT_SEASON = 2024`

En Ligue 1, la saison va d'août à mai de l'année suivante.

## 🔧 Classe FootballAPI

Classe pour interagir facilement avec l'API-Football. La clé API est chargée automatiquement depuis le fichier `.env`.

In [3]:
import time
import os
import hashlib

class FootballAPI:
    """Classe pour interagir avec l'API-Football avec système de cache sur disque"""
    
    def __init__(self, api_key, league_id=61, season=2026, cache_duration=3600, cache_dir='cache'):
        self.base_url = "https://v3.football.api-sports.io"
        self.headers = {'x-apisports-key': api_key}
        self.league_id = league_id
        self.season = season
        self.cache_duration = cache_duration  # Durée du cache en secondes (1h par défaut)
        self.cache_dir = cache_dir
        self.api_calls = 0  # Compteur d'appels API
        
        # Créer le dossier cache s'il n'existe pas
        if not os.path.exists(self.cache_dir):
            os.makedirs(self.cache_dir)
            print(f"📁 Dossier cache créé: {self.cache_dir}")
    
    def _get_cache_key(self, endpoint, params):
        """Génère une clé unique pour le cache"""
        params_str = '&'.join([f"{k}={v}" for k, v in sorted(params.items())])
        return f"{endpoint}?{params_str}"
    
    def _get_cache_filename(self, cache_key):
        """Génère un nom de fichier sécurisé à partir de la clé de cache"""
        # Utiliser un hash MD5 pour créer un nom de fichier unique et sûr
        hash_key = hashlib.md5(cache_key.encode()).hexdigest()
        return os.path.join(self.cache_dir, f"{hash_key}.json")
    
    def _get_from_cache(self, cache_key):
        """Récupère les données du cache fichier si elles sont encore valides"""
        cache_file = self._get_cache_filename(cache_key)
        
        if os.path.exists(cache_file):
            # Vérifier l'âge du fichier
            file_age = time.time() - os.path.getmtime(cache_file)
            
            if file_age < self.cache_duration:
                try:
                    with open(cache_file, 'r', encoding='utf-8') as f:
                        data = json.load(f)
                    print(f"📦 Données récupérées du cache fichier (âge: {int(file_age/60)}min)")
                    return data
                except Exception as e:
                    print(f"⚠️ Erreur lecture cache: {e}")
                    return None
            else:
                print(f"⏰ Cache expiré (âge: {int(file_age/3600):.1f}h)")
        
        return None
    
    def _save_to_cache(self, cache_key, data):
        """Sauvegarde les données dans un fichier cache"""
        cache_file = self._get_cache_filename(cache_key)
        
        try:
            with open(cache_file, 'w', encoding='utf-8') as f:
                json.dump(data, f, ensure_ascii=False, indent=2)
            print(f"💾 Données sauvegardées dans le cache")
        except Exception as e:
            print(f"⚠️ Erreur sauvegarde cache: {e}")
    
    def _make_request(self, endpoint, params):
        """Fait une requête API avec gestion du cache fichier"""
        cache_key = self._get_cache_key(endpoint, params)
        
        # Vérifier le cache d'abord
        cached_data = self._get_from_cache(cache_key)
        if cached_data is not None:
            return cached_data
        
        # Si pas en cache, faire l'appel API
        url = f"{self.base_url}{endpoint}"
        response = requests.get(url, headers=self.headers, params=params)
        self.api_calls += 1
        print(f"🌐 Appel API #{self.api_calls}")
        
        if response.status_code == 200:
            data = response.json()
            self._save_to_cache(cache_key, data)
            return data
        else:
            print(f"❌ Erreur API: {response.status_code}")
        return None
    
    def get_fixtures(self, date=None, team_id=None):
        """Récupère les matchs de la Ligue 1"""
        params = {'league': self.league_id, 'season': self.season}
        
        if date:
            params['date'] = date  # Format: YYYY-MM-DD
        if team_id:
            params['team'] = team_id
        
        return self._make_request('/fixtures', params)
    
    def get_team_statistics(self, team_id):
        """Récupère les statistiques d'une équipe"""
        params = {
            'league': self.league_id,
            'season': self.season,
            'team': team_id
        }
        return self._make_request('/teams/statistics', params)
    
    def get_head_to_head(self, team1_id, team2_id, last=10):
        """Récupère l'historique des confrontations"""
        params = {'h2h': f"{team1_id}-{team2_id}", 'last': last}
        return self._make_request('/fixtures/headtohead', params)
    
    def get_standings(self):
        """Récupère le classement de la Ligue 1"""
        params = {'league': self.league_id, 'season': self.season}
        return self._make_request('/standings', params)
    
    def get_teams(self):
        """Récupère la liste des équipes de la Ligue 1"""
        params = {'league': self.league_id, 'season': self.season}
        return self._make_request('/teams', params)
    
    def clear_cache(self):
        """Vide le cache en supprimant tous les fichiers"""
        if os.path.exists(self.cache_dir):
            files = os.listdir(self.cache_dir)
            for file in files:
                if file.endswith('.json'):
                    os.remove(os.path.join(self.cache_dir, file))
            print(f"🗑️ Cache vidé ({len(files)} fichiers supprimés)")
        else:
            print("📁 Dossier cache inexistant")
    
    def get_cache_stats(self):
        """Affiche les statistiques du cache"""
        if os.path.exists(self.cache_dir):
            files = [f for f in os.listdir(self.cache_dir) if f.endswith('.json')]
            total_size = sum(os.path.getsize(os.path.join(self.cache_dir, f)) for f in files)
            
            print(f"📊 Statistiques du cache:")
            print(f"   - Fichiers en cache: {len(files)}")
            print(f"   - Taille totale: {total_size / 1024:.2f} KB")
            print(f"   - Appels API effectués: {self.api_calls}")
            print(f"   - Durée de validité: {self.cache_duration}s ({self.cache_duration/3600:.1f}h)")
            print(f"   - Dossier: {os.path.abspath(self.cache_dir)}")
        else:
            print(f"📁 Dossier cache inexistant")

# Initialisation de l'API avec cache (1 heure par défaut)
api = FootballAPI(API_KEY, LIGUE1_ID, CURRENT_SEASON, cache_duration=3600)
print("✅ API initialisée avec système de cache sur disque")

✅ API initialisée avec système de cache sur disque


## 💾 Gestion du cache

Le système de cache permet d'économiser les appels API en sauvegardant les données dans le dossier `cache/` :
- Les données sont stockées dans des fichiers JSON pendant 1 heure par défaut
- Affichage de l'origine des données (cache 📦 ou API 🌐)
- Compteur d'appels API réels
- Cache persistant entre les sessions du notebook

In [4]:
# Afficher les statistiques du cache
api.get_cache_stats()

📊 Statistiques du cache:
   - Fichiers en cache: 27
   - Taille totale: 156.38 KB
   - Appels API effectués: 0
   - Durée de validité: 3600s (1.0h)
   - Dossier: c:\Users\nboud\Documents\GitHub-My-Projects\predictions-football-fr-python\cache


In [5]:
# Vider le cache pour forcer une mise à jour depuis l'API
# api.clear_cache()
print("\n💡 Relancez maintenant les cellules de données (statistiques, classement, etc.)")


💡 Relancez maintenant les cellules de données (statistiques, classement, etc.)


## 📈 Statistiques d'une équipe

Exemple avec le PSG (ID: 85), Monaco (ID: 91), Marseille (ID: 81)

In [6]:
# Statistiques d'une équipe (exemple: PSG = 85)
team_id = 85  # Changez l'ID pour une autre équipe

team_stats = api.get_team_statistics(team_id)

if team_stats and team_stats['response']:
    stats = team_stats['response']
    team_name = stats['team']['name']
    
    print(f"📊 Statistiques de {team_name}\n")
    print(f"🎯 Buts marqués: {stats['goals']['for']['total']['total']}")
    print(f"🥅 Buts encaissés: {stats['goals']['against']['total']['total']}")
    
    # Vérifier si la forme est disponible
    form = stats.get('form')
    if form:
        print(f"📍 Forme récente: {form}")
    else:
        print(f"📍 Forme récente: Non disponible")
    
    print(f"🏠 Victoires à domicile: {stats['fixtures']['wins']['home']}")
    print(f"✈️ Victoires à l'extérieur: {stats['fixtures']['wins']['away']}")
    print(f"🤝 Matchs nuls: {stats['fixtures']['draws']['total']}")
    
    # Analyse de la forme (si disponible)
    if form:
        wins = form.count('W')
        draws = form.count('D')
        losses = form.count('L')
        print(f"\n📉 Derniers {len(form)} matchs: {wins}V - {draws}N - {losses}D")
else:
    print("❌ Impossible de récupérer les statistiques")

📦 Données récupérées du cache fichier (âge: 9min)
📊 Statistiques de Paris Saint Germain

🎯 Buts marqués: 43
🥅 Buts encaissés: 16
📍 Forme récente: WWWWLWDDWDWWWLWWWWWW
🏠 Victoires à domicile: 8
✈️ Victoires à l'extérieur: 7
🤝 Matchs nuls: 3

📉 Derniers 20 matchs: 15V - 3N - 2D


In [7]:
# Test rapide: Vérifier la configuration
print("✅ Notebook prêt à l'emploi!")
print("\n💡 N'oubliez pas:")
print("   1. Configurer votre API_FOOTBALL_KEY dans le fichier .env")
print("   2. Vous avez 100 requêtes/jour avec le plan gratuit")
print("   3. Le système de cache économise automatiquement vos appels API")
print("   4. Exécutez les cellules dans l'ordre pour découvrir les fonctionnalités")

✅ Notebook prêt à l'emploi!

💡 N'oubliez pas:
   1. Configurer votre API_FOOTBALL_KEY dans le fichier .env
   2. Vous avez 100 requêtes/jour avec le plan gratuit
   3. Le système de cache économise automatiquement vos appels API
   4. Exécutez les cellules dans l'ordre pour découvrir les fonctionnalités


## 🎯 Prédiction Simple basée sur les statistiques

Exemple de fonction de prédiction basique utilisant la forme récente et les statistiques

In [8]:
def calculate_form_score(form_string):
    """Calcule un score basé sur la forme récente (W=3, D=1, L=0)"""
    if not form_string:
        return 0
    score = 0
    for result in form_string:
        if result == 'W':
            score += 3
        elif result == 'D':
            score += 1
    return score

def simple_prediction(team1_id, team2_id):
    """Prédiction simple basée sur les statistiques"""
    
    # Récupérer les statistiques des deux équipes
    stats1 = api.get_team_statistics(team1_id)
    stats2 = api.get_team_statistics(team2_id)
    
    if not stats1 or not stats2:
        return "❌ Impossible de faire une prédiction"
    
    team1 = stats1['response']
    team2 = stats2['response']
    
    # Calcul des scores (utiliser .get() pour gérer les valeurs None)
    form1 = calculate_form_score(team1.get('form'))
    form2 = calculate_form_score(team2.get('form'))
    
    goals1 = team1['goals']['for']['total']['total']
    goals2 = team2['goals']['for']['total']['total']
    
    conceded1 = team1['goals']['against']['total']['total']
    conceded2 = team2['goals']['against']['total']['total']
    
    # Score composite (forme + buts marqués - buts encaissés)
    score1 = form1 + (goals1 - conceded1) * 0.5
    score2 = form2 + (goals2 - conceded2) * 0.5
    
    # Affichage
    print(f"⚔️  {team1['team']['name']} vs {team2['team']['name']}\n")
    print(f"📊 Analyse:")
    print(f"   {team1['team']['name']}: Forme={form1}, Buts={goals1}, Encaissés={conceded1}")
    print(f"   {team2['team']['name']}: Forme={form2}, Buts={goals2}, Encaissés={conceded2}")
    print(f"\n💯 Scores de prédiction:")
    print(f"   {team1['team']['name']}: {score1:.1f}")
    print(f"   {team2['team']['name']}: {score2:.1f}")
    
    # Prédiction
    if score1 > score2 + 2:
        prediction = f"🏆 Victoire probable de {team1['team']['name']}"
    elif score2 > score1 + 2:
        prediction = f"🏆 Victoire probable de {team2['team']['name']}"
    else:
        prediction = "🤝 Match serré, résultat incertain"
    
    print(f"\n{prediction}")
    
    return prediction

# Exemple de prédiction: PSG vs Marseille
simple_prediction(85, 81)

📦 Données récupérées du cache fichier (âge: 9min)
📦 Données récupérées du cache fichier (âge: 9min)
⚔️  Paris Saint Germain vs Marseille

📊 Analyse:
   Paris Saint Germain: Forme=48, Buts=43, Encaissés=16
   Marseille: Forme=39, Buts=46, Encaissés=22

💯 Scores de prédiction:
   Paris Saint Germain: 61.5
   Marseille: 51.0

🏆 Victoire probable de Paris Saint Germain


'🏆 Victoire probable de Paris Saint Germain'

## 🔄 Confrontations directes (Head-to-Head)

Analyser l'historique entre deux équipes